In [1]:
from pathos.pools import ProcessPool as Pool
import selenium
from selenium import webdriver
import os
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
from bs4 import BeautifulSoup
import random
from random import randint
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import WebDriverException

In [9]:


# part 2 : 페이지 순차 접속 후 게시 정보, 본문 내용, 그리고 댓글 정보 수집하기 
index_list = []
index_no = 1

title_list = []
writer_list = []
posting_date_list = []
contents_list = []

cmt_index_list = []
cmt_nick_name_list = []
cmt_contents_list = []
cmt_time_list = []

cmt_no_list = []
reply_index_list = []
reply_nick_name_list = []
reply_contents_list = []
reply_date_list = []

exception_list = []

def web(site) :
    global index_list
    global index_no
    global title_list
    global writer_list
    global posting_date_list
    global contents_list
    global cmt_index_list
    global cmt_nick_name_list
    global cmt_contents_list
    global cmt_time_list
    global exception_list

    # 크롬 드라이버 호출
    driver_path = '/Users/kibeomkim/Desktop/chromedriver_intel/m1_chromedriver'
    options = webdriver.ChromeOptions()
    options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')
    #options.add_argument('headless')
    #options.add_argument('disable-gpu')
    driver = webdriver.Chrome(driver_path, options=options)

    # 페이지 접속 
    try : 
        driver.set_page_load_timeout(20)
        driver.get(site)
        time.sleep(1.5)
    except TimeoutException : 
        print('로드 시간 초과')
        exception_list.append(site)
        driver.quit()
    except WebDriverException : 
        print('페이지 로드 실패')
        exception_list.append(site)
        driver.quit()
    except StaleElementReferenceException : 
        print('페이지 태그 찾기 실패')
        exception_list.append(site)
        driver.quit()
    except : 
        print('에러 원인 불명, 다음 url 접속')
        exception_list.append(site)
        driver.quit()
        

    # 페이지 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 제대로 된 페이지인지 검사 
    try : 
        soup.find('span', 'title_subject').get_text()
        pass
    except : # 찾을 수 없는 페이지이면 건너뛰기 
        driver.quit()

    # 게시물 인덱스 넘버 추가 
    index_list.append(index_no)

    # 게시글 제목 수집 
    title = soup.find('span', 'title_subject').get_text().replace(' ', "")
    title_list.append(title)
    print('-'*100)
    print(f'게시글 제목 : {title}')

    # 게시자 수집 
    writer = soup.find('span', 'nickname in').get_text().replace(' ', "")
    writer_list.append(writer)
    print(f'게시자 : {writer}')

    # 게시 일자 수집 
    posting_date = soup.find('span', 'gall_date').get_text().replace(' ', "")
    posting_date_list.append(posting_date)
    print(f'게시 일자 : {posting_date}')

    # 본문 내용 수집 
    contents = soup.find('div', 'write_div').get_text().replace('\n', "")
    contents_list.append(contents)
    print(f'본문 내용 : {contents}')

    for scroll in range(2) : # 스크롤 다운
        driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1.5)

    # 댓글 & 대댓글 수집 
    try : # 댓글이 있으면 
        driver.find_element_by_class_name('cmt_list')
        
        cmt_list = soup.find('ul', 'cmt_list').find_all('li', 'ub-content')
        print('*'*100)

        for li in cmt_list : 

            cmt_index_list.append(index_no) # 어느 게시글 댓글인지 표시 

            # 작성자 닉네임 수집
            try : 
                cmt_nick_name = li.find('div', 'cmt_nickbox').get_text().replace('\n', '').replace(' ', '')
                if len(cmt_nick_name) == 0 : # 작성자 이름 태그 있는데, 작성자 이름도 있는지 없는지 검사 
                    cmt_nick_name_list.append(' ')
                    print(f'댓글 작성자 닉네임 : 정보 없음')
                else : 
                    cmt_nick_name_list.append(cmt_nick_name)
                    print(f'댓글 작성자 닉네임 : {cmt_nick_name}')
            except : 
                cmt_nick_name_list.append(' ')
                print(f'댓글 작성자 태그 없음')
            
            # 댓글 내용 수집 
            # 댓글 내용 태그 있는지 검사 
            try : # 일반 댓글인가 
                cmt_contents = li.find('div','clear cmt_txtbox btn_reply_write_all').get_text().replace('\n', '').replace(' ', '')
                if len(cmt_contents) == 0 : 
                    cmt_contents_list.append(' ')
                    print('댓글내용 : 정보 없음')
                else : 
                    cmt_contents_list.append(cmt_contents)
                    print(f'댓글내용 : {cmt_contents}')
            except : 
                try : # 대댓글인가
                    cmt_contents = li.find('div', 'clear cmt_txtbox').get_text().replace('\n', "").replace(' ', '')
                    if len(cmt_contents) == 0 : 
                        cmt_contents_list.append(' ')
                        print('댓글내용 : 정보 없음')
                    else : 
                        cmt_contents_list.append(cmt_contents)
                        print(f'댓글내용 : {cmt_contents}')
                except : 
                    cmt_contents_list.append(' ')
                    print('댓글 내용 태그 없음')
                

            # 댓글 게시 시간 수집 
            try : 
                cmt_time = li.find('span', 'date_time').get_text().replace('\n', '')
                if len(cmt_time) == 0 : 
                    cmt_time_list.append(' ')
                    print(f'댓글내용 : 정보 없음')
                else : 
                    cmt_time_list.append(cmt_time)
                    print(f'댓글 게시 시간 : {cmt_time}')
            except : 
                cmt_time_list.append(' ')
                print(f'댓글 게시 시간 태그 없음')

        print('*'*100) # 댓글 닫기 
        print('-'*100) # 본문 닫기 
        index_no += 1
        driver.quit() 
        
    except : # 댓글이 없으면 
        print('댓글 없음')
        print('-'*100) # 본문 닫기 

        index_no += 1
        driver.quit() # 댓글 없다. 다음 페이지로 가자. 


In [7]:
f = open('/Users/kibeomkim/Desktop/접속주소_테스트 2.txt')
ll = f.readlines()

new_list = []
for i in ll : 
    i = i.replace('\n', '')
    i = 'https://gall.dcinside.com/'+i
    new_list.append(i)